### Algorytm Branch-and-bound

Kanały komunikacyjne pomiędzy zadaniami współbieżnymi (tutaj w ramach jednego procesu)

In [1]:
# Definijemy kanał komunikacyjny
c= Channel(32)

# funkcja consumer czyta dane z kanalu komunikacyjnego 
function consumer()
    for i in 1:3
        data = take!(c)
        println("Konsumuje ", data)
    end    
end

#uruchamiam funkcje consumer() jako współbieżne zadanie (Task)
consumertask= @schedule consumer()


Task (runnable) @0x00007f6cbe131f90

In [2]:
# pętla producenta
for i in 1:3
    put!(c,i) 
end

Konsumuje 1
Konsumuje 2
Konsumuje 3


Przykład użycia zdalnych kanałow komunikacji (Remote Channels)

In [3]:
addprocs(4)
nprocs()

5

In [4]:
# zdalny kanał przekazujący numery zadań
const jobs = RemoteChannel(()->Channel{Int}(32));

In [5]:
# zdalny kanał przekazujący wyniki
const results = RemoteChannel(()->Channel{Tuple}(32));

In [6]:
# funkcja wykonująca pewną (symulowaną sleepem) "pracę"
@everywhere function do_work(jobs, results) 
           while true
               job_id = take!(jobs)
               exec_time = rand()
               sleep(exec_time) # symulacja "pracy"
               put!(results, (job_id, exec_time, myid()))
           end
       end

In [7]:
# funkcja wkładająca  do kanału numery "prac"
function make_jobs(n)
           for i in 1:n
               put!(jobs, i)
           end
end;

In [8]:
n = 12;

In [9]:
@schedule make_jobs(n);

In [10]:
# starujemy zdalne zadania
for p in workers() 
           @async remote_do(do_work, p, jobs, results)
end


In [11]:
# wypisujemy wyniki
@elapsed while n > 0 
           job_id, exec_time, where = take!(results)
           println("$job_id finished in $(round(exec_time,2)) seconds on worker $where")
           n = n - 1
       end

3 finished in 0.27 seconds on worker 3
4 finished in 0.35 seconds on worker 2
1 finished in 0.88 seconds on worker 4
6 finished in 0.49 seconds on worker 2
2 finished in 0.96 seconds on worker 5
8 finished in 0.25 seconds on worker 2
5 finished in 0.94 seconds on worker 3
11 finished in 0.05 seconds on worker 3
7 finished in 0.56 seconds on worker 4
12 finished in 0.22 seconds on worker 3
9 finished in 0.63 seconds on worker 5
10 finished in 0.92 seconds on worker 2


3.662274086

### Opis problemu

* Znaną klasą problemów sa zadania decyzyjne. Przykladem takiego problemu moze być <a href="http://www.mcs.anl.gov/~itf/dbpp/text/node21.html#SECTION02370000000000000000">optymalizacja zagospodarownia miejsca w okreslonym obszarze potrzebna m.in w projektowaniu ukladow VLSI.</a>

* Powyższy link pokazuje jeden ze sposobow zrownoleglania takiego problemu za pomoca drzewa i algorymu "branch-and-bound search". Zadanie ma na celu zapoznianie sie z tym algorytmem i zastosowanie go do problemu decyzyjnego na przykladzie problemu komiwłojażera

* Przyklad: Problem komiwłojażera (travelling salesman problem)
Dane jest N miast oraz dlugosci odcinkow pomiedzy miastami, należy znalezc cykl o minimalnej dlugosci zawierajacy wszystkie miasta. 

* Uwaga: użycie algorytmu branch-and-bound jest obowiązkowe dla tego ćwiczenia! 



### Zadanie


* Przestawić krótki opis algorytmu równoległego branch and bound wg metodologii PCAM
* Zaimplementować rozwiązanie  w jezyku Julia za pomocą mechanizmu przydziału prac do workerów. Kolejne poddrzewa są kolejnymi "pracami".
* Należy zmierzyć jaka gruboziarnistość poddrzew jest optymalna dla zaproponowanego rozwiązania. Czy dużo cienkich poddrzew, czy raczej mało grubych poddrzew? Przedstawić wykresy przyspieszenia i efektywności.
* Workerzy  komunikują aktualne minimum za pomocą mechanizmów  channeli. Na tej podstawie dokonywane jest obcinanie drzew (pruning). 


